# Demo 7 - Migrate vector app

In [ ]:
!pip install --quiet "astrapy>=1.0.0" "python-dotenv>=1.0.0" "chromadb>=0.4.21"

In [ ]:
import getpass
import os

from dotenv import load_dotenv

from astrapy import DataAPIClient
from astrapy.constants import VectorMetric

## ChromaDB "app"

In [ ]:
import chromadb
chroma_client = chromadb.Client()

# Creating a Vector store
cos_coll = chroma_client.create_collection(
    name="cosine_coll",
    metadata={"hnsw:space": "cosine"},
)

# Saving vector entries
cos_coll.add(
    documents=["3 o-clock", "6 o'clock", "9 o'clock"],
    embeddings=[[1, 0], [0, -1], [-1, 0]],
    ids=["3:00", "6:00", "9:00"],
)

# Running ANN search
cos_matches = cos_coll.query(
    query_embeddings=[[1, 0]],
    n_results=3
)

chroma_threshold = 1.5

# Printing the results and their "distance"
match_ids = cos_matches["ids"][0]
match_distances = cos_matches["distances"][0]
for m_id, m_distance in zip(match_ids, match_distances):
    status = "ok" if m_distance <= chroma_threshold else "NO!"
    print(f"d_chroma,Cos([1,0], '{m_id})' = {m_distance}: {status}")

## Setup (Astra) DB

In [ ]:
load_dotenv()

if "ASTRA_DB_APPLICATION_TOKEN" not in os.environ:
    os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass.getpass("Please input your Astra DB Token:")

if "ASTRA_DB_API_ENDPOINT" not in os.environ:
    os.environ["ASTRA_DB_API_ENDPOINT"] = input("Please input your Astra DB API Endpoint:")

if "ASTRA_DB_KEYSPACE" not in os.environ:
    _namespace = input("(Optional) Input your Astra DB namespace if desired, or leave blank:")
    if _namespace:
        os.environ["ASTRA_DB_KEYSPACE"] = _namespace

ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]
ASTRA_DB_API_ENDPOINT = os.environ["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_KEYSPACE = os.environ.get("ASTRA_DB_KEYSPACE")

In [ ]:
db = DataAPIClient(ASTRA_DB_APPLICATION_TOKEN).get_database_by_api_endpoint(ASTRA_DB_API_ENDPOINT, namespace=ASTRA_DB_KEYSPACE)

## Migrated Astra DB "app"

In [ ]:
app_collection = db.create_collection(
    "migrated_app",
    dimension=2,
    metric=VectorMetric.COSINE,
    check_exists=False,
)
# just in case this demo is re-run
app_collection.delete_all()

In [ ]:
app_collection.insert_many([
    {"name": "3:00", "$vector": [1, 0]},
    {"name": "6:00", "$vector": [0, -1]},
    {"name": "9:00", "$vector": [-1, 0]},
])

In [ ]:
astra_threshold = 1 - chroma_threshold / 2

# Printing the results and their "similarity"
for match in app_collection.find(vector=[1, 0], projection={"*": True}, include_similarity=True):
    # (just for more illustrative outputs, recast it also to a Chroma distance)
    chroma_dist = 2 * (1 - match["$similarity"])
    #
    status = "ok" if match["$similarity"] > astra_threshold else "NO!"
    print(
        f"S_cos([1,0], '{match['name']})' = {match['$similarity']}: {status} "
        f"(d_chroma,Cos = {chroma_dist})"
    )

## Cleanup

In [ ]:
app_collection.delete_all()

# free all resources with:
# app_collection.drop()

## The End